In [22]:
from pynq import Overlay, GPIO
from time import sleep

# 載入 bitstream
overlay = Overlay("./ps_gpio_kv260.bit")

In [ ]:

# GPIO
dir_led    = GPIO(GPIO.get_gpio_pin(0), 'out')
move_led   = GPIO(GPIO.get_gpio_pin(1), 'out')
door_led   = GPIO(GPIO.get_gpio_pin(2), 'out')
floor_led1 = GPIO(GPIO.get_gpio_pin(3), 'out')
floor_led2 = GPIO(GPIO.get_gpio_pin(4), 'out')
bit0 = GPIO(GPIO.get_gpio_pin(5), 'in')
bit1 = GPIO(GPIO.get_gpio_pin(6), 'in')
bit2 = GPIO(GPIO.get_gpio_pin(7), 'in')

# 狀態
floor_pos = 0
direction = 1
requests  = [0,0,0,0]
EMERGENCY = False
last_req  = [0,0,0,0]

# ---------- 顯示樓層 ----------
def show_floor(pos):
    if pos<100:   floor_led1.write(0); floor_led2.write(0); return 0
    if pos<200:   floor_led1.write(1); floor_led2.write(0); return 1
    if pos<300:   floor_led1.write(0); floor_led2.write(1); return 2
    floor_led1.write(1); floor_led2.write(1); return 3

# ---------- 按鈕 ----------
def read_buttons():
    global EMERGENCY, last_req
    code = f"{bit2.read()}{bit1.read()}{bit0.read()}"
    if   code=='001': requests[0]=1
    elif code=='010': requests[1]=1
    elif code=='011': requests[2]=1
    elif code=='100': requests[3]=1
    elif code=='101':
        EMERGENCY=True
        for i in range(1,4): requests[i]=0
        requests[0]=1
        print("‼ 緊急呼叫 (清空其他請求)")

    if requests != last_req:
        print("🆕 請求更新:", requests)
        last_req = requests.copy()

# ---------- 門 ----------
def door_cycle():
    door_led.write(1); print("🚪 開門")
    sleep(2)
    door_led.write(0); print("🚪 關門")

# ---------- 一單位移動 ----------
def move_unit(step, start_pos, target_pos):
    """
    step: +1 / -1
    start_pos: 起點絕對位置
    target_pos: 目標絕對位置
    return 到達樓層 idx 或 None
    """
    global floor_pos
    progress = abs(floor_pos - start_pos)        # 已走距離
    remain   = abs(target_pos - floor_pos)       # 剩餘距離
    is_down  = step == -1

    head_slow = 10 if is_down else 10
    tail_slow = 10 if is_down else 10

    slow = (progress < head_slow) or (remain <= tail_slow)
    delay = 0.6 if slow else 0.1

    move_led.write(1); sleep(delay/2)
    move_led.write(0); sleep(delay/2)

    floor_pos += step
    floor_idx = floor_pos//100 if floor_pos%100==0 else None
    show_floor(floor_pos)
    return floor_idx

# ---------- 旅行 ----------
def travel_to(dest):
    global floor_pos
    step = 1 if dest > floor_pos//100 else -1
    start_pos  = floor_pos
    target_pos = dest*100
    print(f"➡ 前往 {dest+1}F")
    while floor_pos != target_pos:
        read_buttons()
        if EMERGENCY: print("↩ 被緊急打斷"); return False
        fl = move_unit(step, start_pos, target_pos)
        if fl is not None and requests[fl]:
            print(f"★ 停靠 {fl+1}F")
            requests[fl]=0
            door_cycle()
            return True
    print(f"★ 到達 {dest+1}F")
    door_cycle()
    return True

# ---------- FSM ----------
def fsm_once():
    global direction, EMERGENCY
    read_buttons()
    cur = floor_pos//100

    if EMERGENCY:                       # 緊急最高優先
        direction=0; dir_led.write(0)
        travel_to(0); EMERGENCY=False; return

    if direction:                       # 上行
        for f in range(cur,4):
            if requests[f]:
                dir_led.write(1)
                travel_to(f); return
        direction=0
    else:                               # 下行
        for f in range(cur-1,-1,-1):
            if requests[f]:
                dir_led.write(0)
                travel_to(f); return
        direction=1

print("🚀 Elevator controller (balanced slow start & stop) START")
while True:
    fsm_once()
    sleep(0.05)


🚀 Elevator controller (balanced slow start & stop) START
🆕 請求更新: [0, 0, 0, 1]
➡ 前往 4F
★ 停靠 4F
🚪 開門
🚪 關門
🆕 請求更新: [0, 0, 0, 0]
🆕 請求更新: [0, 1, 0, 0]
➡ 前往 2F
🆕 請求更新: [1, 1, 0, 0]
★ 停靠 2F
🚪 開門
🚪 關門
🆕 請求更新: [1, 0, 0, 0]
➡ 前往 1F
★ 停靠 1F
🚪 開門
🚪 關門
🆕 請求更新: [0, 0, 0, 0]
🆕 請求更新: [0, 0, 1, 0]
➡ 前往 3F
‼ 緊急呼叫 (清空其他請求)
🆕 請求更新: [1, 0, 0, 0]
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩ 被緊急打斷
‼ 緊急呼叫 (清空其他請求)
➡ 前往 1F
‼ 緊急呼叫 (清空其他請求)
↩